In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 35.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 63.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 77.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.7 MB/s eta 0:00:0

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
label_type = 'ner'

In [ ]:
columns={0:'text',1:'pos',2:'ner'}

In [ ]:
datafolder='/content/drive/MyDrive/Subinay/7_tags/'
from flair.datasets import ColumnCorpus
from flair.data import Corpus

In [ ]:
corpus: Corpus = ColumnCorpus(datafolder, columns,
                              train_file='tag_sequence_without_pseudo_flair.txt',test_file='tag_sequence_test_7tags_flair.txt',dev_file='tag_sequence_dev_8tags_flair.txt'
                  )

2023-06-21 16:15:46,326 Reading data from /content/drive/MyDrive/Subinay/7_tags
2023-06-21 16:15:46,329 Train: /content/drive/MyDrive/Subinay/7_tags/tag_sequence_without_pseudo_flair.txt
2023-06-21 16:15:46,335 Dev: /content/drive/MyDrive/Subinay/7_tags/tag_sequence_dev_8tags_flair.txt
2023-06-21 16:15:46,337 Test: /content/drive/MyDrive/Subinay/7_tags/tag_sequence_test_7tags_flair.txt


In [ ]:
label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)

2023-06-21 16:15:50,368 Computing label dictionary. Progress:


262it [00:00, 5520.57it/s]

2023-06-21 16:15:50,427 Dictionary created for label 'ner' with 8 values: Otag (seen 4808 times), homicide (seen 1043 times), riot (seen 797 times), assault (seen 734 times), evidence (seen 580 times), wittest (seen 560 times), imprisonment (seen 509 times), expwittest (seen 502 times)


In [ ]:
print(label_dict)

Dictionary with 8 tags: Otag, evidence, riot, assault, wittest, imprisonment, expwittest, homicide


In [ ]:
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

In [ ]:
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

2023-06-19 06:45:25,497 SequenceTagger predicts: Dictionary with 8 tags: Otag, evidence, riot, assault, wittest, imprisonment, expwittest, homicide


In [ ]:
trainer = ModelTrainer(tagger, corpus)

In [ ]:
trainer.fine_tune('/content/drive/MyDrive/Subinay/5_riot_10_evidence_gpt2_pr/',
                  learning_rate=5.0e-6,
                  mini_batch_size=3,
                  #mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  )

2023-06-19 07:41:11,318 ----------------------------------------------------------------------------------------------------
2023-06-19 07:41:11,323 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250003, 1024)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out

100%|██████████| 4/4 [00:00<00:00,  5.60it/s]

2023-06-19 07:42:00,398 Evaluating as a multi-label problem: False
2023-06-19 07:42:00,416 DEV : loss 1.5980676412582397 - f1-score (micro avg)  0.6405
2023-06-19 07:42:00,423 ----------------------------------------------------------------------------------------------------


2023-06-19 07:42:04,769 epoch 2 - iter 8/84 - loss 0.12862250 - time (sec): 4.34 - samples/sec: 146.88 - lr: 0.000005
2023-06-19 07:42:09,694 epoch 2 - iter 16/84 - loss 0.17565832 - time (sec): 9.27 - samples/sec: 137.78 - lr: 0.000005
2023-06-19 07:42:14,416 epoch 2 - iter 24/84 - loss 0.20688917 - time (sec): 13.99 - samples/sec: 149.25 - lr: 0.000005
2023-06-19 07:42:19,372 epoch 2 - iter 32/84 - loss 0.18921568 - time (sec): 18.95 - samples/sec: 157.18 - lr: 0.000005
2023-06-19 07:42:24,078 epoch 2 - iter 40/84 - loss 0.19339085 - time (sec): 23.65 - samples/sec: 160.96 - lr: 0.000005
2023-06-19 07:42:29,713 epoch 2 - iter 48/84 - loss 0.19750709 - time (sec): 29.29 - samples/sec: 161.95 - lr: 0.000005
2023-06-19 07:42:34,581 epoch 2 - iter 56/84 - loss 0.24840675 - time (sec): 34.16 - samples/sec: 165.48 - lr: 0.000005
2023-06-19 07:42:39,378 epoch 2 - iter 64/84 - loss 0.22591290 - time (sec): 38.95 - samples/sec: 167.72 - lr: 0.000005
2023-06-19 07:42:44,227 epoch 2 - iter 72/8

100%|██████████| 4/4 [00:00<00:00,  5.39it/s]

...

2023-06-19 07:49:51,207 Evaluating as a multi-label problem: False
2023-06-19 07:49:51,232 DEV : loss 1.472876787185669 - f1-score (micro avg)  0.6314


2023-06-19 07:50:05,331 ----------------------------------------------------------------------------------------------------
2023-06-19 07:50:05,339 Testing using last state of model ...


100%|██████████| 246/246 [00:42<00:00,  5.84it/s]


2023-06-19 07:50:47,539 Evaluating as a multi-label problem: False
2023-06-19 07:50:47,745 0.6436	0.6436	0.6436	0.6436
2023-06-19 07:50:47,746 
Results:
- F-score (micro) 0.6436
- F-score (macro) 0.5017
- Accuracy 0.6436

By class:
              precision    recall  f1-score   support

  expwittest     0.9741    0.7594    0.8535      7789
        Otag     0.5811    0.8133    0.6779      4918
     wittest     0.7568    0.2252    0.3471      3841
    evidence     0.1292    0.9689    0.2280       257
     assault     0.4662    0.8503    0.6023       755
        riot     0.3676    0.9585    0.5314       410
    homicide     0.4190    0.1747    0.2466      1036
imprisonment     0.4664    0.6047    0.5266       344

    accuracy                         0.6436     19350
   macro avg     0.5201    0.6694    0.5017     19350
weighted avg     0.7484    0.6436    0.6451     19350

2023-06-19 07:50:47,753 ---------------------------------------------------------------------------------------------

{'test_score': 0.6435658914728682,
 'dev_score_history': [0.6404833836858006,
  0.5981873111782477,
  0.6435045317220544,
  0.6525679758308157,
  0.5891238670694864,
  0.5800604229607251,
  0.6132930513595166,
  0.5981873111782477,
  0.6132930513595166,
  0.6314199395770392],
 'train_loss_history': [0.1971358872752552,
  0.31771004038583167,
  0.23542266480729035,
  0.22821419044504682,
  0.20324077769956805,
  0.20716582914055306,
  0.2167189044486571,
  0.1864443739786278,
  0.14386032156813694,
  0.11541792703427584],
 'dev_loss_history': [1.5980676412582397,
  1.4579026699066162,
  1.6676369905471802,
  1.727426528930664,
  1.3737729787826538,
  2.2991256713867188,
  1.668044924736023,
  1.459249496459961,
  1.543781042098999,
  1.472876787185669]}

In [ ]:
model = SequenceTagger.load("/content/drive/MyDrive/Subinay/tagger_gpt3.5_5_riot_10_evidence/final-model.pt")

2023-06-17 11:45:15,536 SequenceTagger predicts: Dictionary with 8 tags: Otag, riot, evidence, assault, wittest, imprisonment, expwittest, homicide
